<a href="https://colab.research.google.com/github/notjaswanth69/fs-8/blob/main/fs_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Load the dataset
data = pd.read_csv('/content/melbourne_housing_raw.csv')

# Encode categorical features
label_encoder = LabelEncoder()
for column in data.select_dtypes(include=['object']).columns:
    data[column] = label_encoder.fit_transform(data[column])

# Define features (X) and target variable (y)
X = data.drop('Propertycount', axis=1)
y = data['Propertycount']

# Handle missing values in the target variable
if y.isnull().any():
    y = y.dropna()
    X = X.loc[y.index]

# Drop any remaining missing values in features
X_dropped = X.dropna()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y[X_dropped.index], test_size=0.3, random_state=42)

def evaluate_model(X_train, X_test, y_train, y_test):
    """Train a Linear Regression model and return the Mean Absolute Error."""
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

def forward_feature_selection(X_train, X_test, y_train, y_test):
    """Perform forward feature selection to find the optimal set of features."""
    selected_features = []
    remaining_features = list(X_train.columns)
    best_mae = float('inf')
    mae_progression = []

    while remaining_features:
        mae_candidates = {}
        for feature in remaining_features:
            current_features = selected_features + [feature]
            mae = evaluate_model(X_train[current_features], X_test[current_features], y_train, y_test)
            mae_candidates[feature] = mae

        # Select the feature with the lowest MAE
        best_feature = min(mae_candidates, key=mae_candidates.get)
        best_feature_mae = mae_candidates[best_feature]

        if best_feature_mae < best_mae:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            best_mae = best_feature_mae
            mae_progression.append(best_mae)
            print(f"Selected feature: {best_feature}, MAE: {best_feature_mae:.4f}")
        else:
            break

    return selected_features, mae_progression

# Perform forward feature selection
selected_features, mae_progression = forward_feature_selection(X_train, X_test, y_train, y_test)

# Display results
print(f"\nOptimal set of features: {selected_features}")
print(f"MAE progression: {mae_progression}")

# Evaluate final performance with all features
final_mae_all_features = evaluate_model(X_train, X_test, y_train, y_test)
print(f"MAE with all features: {final_mae_all_features:.4f}")

# Evaluate final performance with selected features
final_mae_selected_features = evaluate_model(X_train[selected_features], X_test[selected_features], y_train, y_test)
print(f"Final MAE with selected features: {final_mae_selected_features:.4f}")

Selected feature: Regionname, MAE: 3419.9372
Selected feature: Type, MAE: 3388.3608
Selected feature: Postcode, MAE: 3386.3254
Selected feature: Bathroom, MAE: 3383.1496
Selected feature: Distance, MAE: 3381.8061
Selected feature: Date, MAE: 3381.5598

Optimal set of features: ['Regionname', 'Type', 'Postcode', 'Bathroom', 'Distance', 'Date']
MAE progression: [3419.9371552030493, 3388.3608408268474, 3386.325435298081, 3383.149568388873, 3381.806063508436, 3381.559820194518]
MAE with all features: 3462.3589
Final MAE with selected features: 3381.5598
